<a href="https://colab.research.google.com/github/killian31/NLP_Project105/blob/main/Multilingual_machine_reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1


In [1]:
import torch
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")



## 1.1 Preprocessing

In [2]:
!pip install datasets
!pip install spacy
!python -m spacy download ja_core_news_sm 
!python -m spacy download fi_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.4 MB/s 
     |████████████████████████████████| 120 kB 66.7 MB/s 
     |████████████████████████████████| 115 kB 61.9 MB/s 
     |████████████████████████████████| 212 kB 54.0 MB/s 
     |████████████████████████████████| 127 kB 63.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.9 MB/s 
     |████████████████████████████████| 2.2 MB 4.7 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20220729-py3-none-any.whl size=71570387 sha256=58097a2a55950c2b02e26538973ae

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import re
import spacy 
import numpy as np
import pandas as pd
from datasets import load_dataset

In [5]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set_pd = train_set.to_pandas()
df = train_set_pd[train_set_pd.language.isin(['finnish','english','japanese'])]

validation_set_pd = validation_set.to_pandas()
df_val = validation_set_pd[validation_set_pd.language.isin(['finnish','english','japanese'])]

Downloading:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]


### 1.1.a Tokenization of english, japanese and finnish.

In [6]:
# Load relevant languages
nlp_en = spacy.load('en_core_web_sm')
nlp_ja = spacy.load('ja_core_news_sm')
nlp_fi = spacy.load('fi_core_news_sm')

In [7]:
# Function to tokenize text
def tokenizer_lang(text, lang):
  if lang in ['english','japanese','finnish']:
    if lang == 'english':
      doc = nlp_en.tokenizer(text)
    elif lang == 'japanese':
      doc = nlp_ja.tokenizer(text)
    elif lang == 'finnish':
      doc = nlp_fi.tokenizer(text)

    return [t.text for t in doc if not t.is_punct]

In [8]:
def tokenize_col(df, cols):
  for col in cols:
    df.loc[:, col+'_t'] = df.apply(lambda x: tokenizer_lang(x[col], x['language']), axis=1)

In [9]:
tokenize_col(df, ['question_text', 'document_plaintext'])
tokenize_col(df_val, ['question_text', 'document_plaintext'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [10]:
df.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_t,document_plaintext_t
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort,"[Milloin, Charles, Fort, syntyi]","[Charles, Hoy, Fort, 6, elokuuta, joidenkin, l..."
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...,"[ダン, ダニエル, ジャドソン, キャラハン, の, 出身, は, どこ]","[ダン, こと, ダニエル, ジャドソン, キャラハン, は, 1890, 年, 7, 月,..."
10,Mitä on altruismi?,Altruismi,finnish,"{'answer_start': [44], 'answer_text': ['epäits...","\n\n\nAltruismi ([1], ”toinen”[2]) tarkoittaa...",https://fi.wikipedia.org/wiki/Altruismi,"[Mitä, on, altruismi]","[\n\n\n, Altruismi, 1, , toinen”[2, tarkoitta..."
12,Mikä oli Wilhelm Wagner viimeinen sävellys?,Richard Wagner,finnish,"{'answer_start': [597], 'answer_text': ['Parsi...",Wagnerin mestariteoksia ovat hänen myöhäiskaud...,https://fi.wikipedia.org/wiki/Richard%20Wagner,"[Mikä, oli, Wilhelm, Wagner, viimeinen, sävellys]","[Wagnerin, mestariteoksia, ovat, hänen, myöhäi..."
13,Missä Harz sijaitsee?,Harz,finnish,"{'answer_start': [25], 'answer_text': ['Pohjoi...",\n\nHarz on horstivuoristo Pohjois-Saksassa[1]...,https://fi.wikipedia.org/wiki/Harz,"[Missä, Harz, sijaitsee]","[\n\n, Harz, on, horstivuoristo, Pohjois-Saksa..."


In [11]:
df_val.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_t,document_plaintext_t
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...,"[化学, 兵器, 禁止, 条約, は, どこ, で, 採択, さ, れ, た]","[1993, 年, 1, 月, 13, 日, に, パリ, に, おい, て, 署名, が,..."
9,オリヴィア・デ・ハヴィランドが生まれたのはいつ,オリヴィア・デ・ハヴィランド,japanese,"{'answer_start': [46], 'answer_text': ['1916年7...","\nオリヴィア・デ・ハヴィランド（Dame Olivia De Havilland, DBE...",https://ja.wikipedia.org/wiki/%E3%82%AA%E3%83%...,"[オリヴィア, デ, ハヴィランド, が, 生まれ, た, の, は, いつ]","[\n, オリヴィア, デ, ハヴィランド, Dame, Olivia, De Havill..."
11,Kauanko lasia on valmistettu?,Lasi,finnish,"{'answer_start': [160], 'answer_text': ['noin ...",Vanhin tunnettu lasilaatu on alkali­kalkki­las...,https://fi.wikipedia.org/wiki/Lasi,"[Kauanko, lasia, on, valmistettu]","[Vanhin, tunnettu, lasilaatu, on, alkali­kalkk..."
14,Mikä on Ponzi-huijaus?,Ponzi-huijaus,finnish,"{'answer_start': [39], 'answer_text': ['pyrami...",Ponzi-huijaus eli Ponzi-järjestelmä on pyramid...,https://fi.wikipedia.org/wiki/Ponzi-huijaus,"[Mikä, on, Ponzi-huijaus]","[Ponzi-huijaus, eli, Ponzi-järjestelmä, on, py..."
20,Mikä oli Napoleonin sotien lopputulos?,Napoleonin sodat,finnish,"{'answer_start': [15], 'answer_text': ['Ranska...",Sotien jälkeen Ranska alistettiin kovilla rauh...,https://fi.wikipedia.org/wiki/Napoleonin%20sodat,"[Mikä, oli, Napoleonin, sotien, lopputulos]","[Sotien, jälkeen, Ranska, alistettiin, kovilla..."


**Remember to preprocess the document_plaintext tokens list (removing brackets, ...)**

## 1.1.a bis: using bpemb

In [21]:
!pip3 install bpemb
import nltk
nltk.download('stopwords')
import spacy
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from bpemb import BPEmb

# Load english model with 25k word-pieces
# bpemb_en = BPEmb(lang='en', dim=100, vs=25000)
# bpemb_jap = BPEmb(lang='ja', dim=100, vs=25000)
# bpemb_fin = BPEmb(lang='fi', dim=100, vs=25000)

# Load the multilingual model with 1000000 word-pieces
bpemb_multi = BPEmb(lang="multi", dim=300, vs=1000000)

downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.model


100%|██████████| 20636145/20636145 [00:01<00:00, 10866904.68B/s]


downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.d300.w2v.bin.tar.gz


100%|██████████| 1123372891/1123372891 [00:50<00:00, 22325221.23B/s]


In [16]:
df['answerable'] = df.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')
df_val['answerable'] = df_val.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_t,document_plaintext_t,answerable_t,answerable
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort,"[Milloin, Charles, Fort, syntyi]","[Charles, Hoy, Fort, 6, elokuuta, joidenkin, l...",Answerable,Answerable
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...,"[ダン, ダニエル, ジャドソン, キャラハン, の, 出身, は, どこ]","[ダン, こと, ダニエル, ジャドソン, キャラハン, は, 1890, 年, 7, 月,...",Answerable,Answerable
10,Mitä on altruismi?,Altruismi,finnish,"{'answer_start': [44], 'answer_text': ['epäits...","\n\n\nAltruismi ([1], ”toinen”[2]) tarkoittaa...",https://fi.wikipedia.org/wiki/Altruismi,"[Mitä, on, altruismi]","[\n\n\n, Altruismi, 1, , toinen”[2, tarkoitta...",Answerable,Answerable
12,Mikä oli Wilhelm Wagner viimeinen sävellys?,Richard Wagner,finnish,"{'answer_start': [597], 'answer_text': ['Parsi...",Wagnerin mestariteoksia ovat hänen myöhäiskaud...,https://fi.wikipedia.org/wiki/Richard%20Wagner,"[Mikä, oli, Wilhelm, Wagner, viimeinen, sävellys]","[Wagnerin, mestariteoksia, ovat, hänen, myöhäi...",Answerable,Answerable
13,Missä Harz sijaitsee?,Harz,finnish,"{'answer_start': [25], 'answer_text': ['Pohjoi...",\n\nHarz on horstivuoristo Pohjois-Saksassa[1]...,https://fi.wikipedia.org/wiki/Harz,"[Missä, Harz, sijaitsee]","[\n\n, Harz, on, horstivuoristo, Pohjois-Saksa...",Answerable,Answerable


In [20]:
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
labels = {'Non-answerable':0, 'Answerable':1}

def get_bpemb_features(dataset, bpemb):
  # With bpemb we can tokenize and embed an entire document using .embed(x)
  X = [bpemb.embed(x).mean(0) for x in tqdm(dataset[:,0])]
  y = [labels[l] for l in dataset['answerable']]
  return X,y

In [ ]:
def run_classifier(X_train, y_train, X_test, y_test):
  # Define hyperparams for search
  # C = np.logspace(-6, 2, 50)
  # warm_start = [False, True]
  # class_weight = ['balanced', None]

  # hp = {"C": C, "warm_start": warm_start, 'class_weight': class_weight}

  classifier = LogisticRegression(penalty='l2', max_iter=1000)
  # classifier_random = RandomizedSearchCV(
  #     estimator=classifier,
  #     param_distributions=hp,
  #     n_iter=100,
  #     cv=5,
  #     verbose=2,
  #     random_state=1000,
  #     n_jobs=-1,
  #     scoring='f1'
  # )

  # classifier_random.fit(X_train, y_train)
  # print(classifier_random.best_params_)
  # print(classifier_random.best_score_)
  # model = classifier_random.best_estimator_
  classifier.fit(X_train,y_train)
  preds = classifier.predict(X_test)
  print(classification_report(y_test, preds))

### 1.1.b Identify most common first and last words

In [12]:
df.loc[:,'first_token'] = df['question_text_t'].apply(lambda x: x[0])
df.loc[:,'last_token'] = df['question_text_t'].apply(lambda x: x[-1])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [13]:
print('Most common first word:')
print(df.groupby(['language'])['first_token'].agg(pd.Series.mode))

print('\nFive most common first words:')
print(df.groupby(['language'])['first_token'].head(5))

Most common first word:
language
english        When
finnish     Milloin
japanese         日本
Name: first_token, dtype: object

Five most common first words:
0      Milloin
1           ダン
10        Mitä
12        Mikä
13       Missä
15       Miten
17          サー
26        When
43         Who
45         ラオス
53          日本
66          西村
112       When
123        Who
125       What
Name: first_token, dtype: object


In [14]:
print('Most common last word:')
print(df.groupby(['language'])['last_token'].agg(pd.Series.mode))

print('\nFive most common last words:')
print(df.groupby(['language'])['last_token'].head(5))

Most common last word:
language
english       born
finnish     syntyi
japanese         た
Name: last_token, dtype: object

Five most common last words:
0          syntyi
1              どこ
10      altruismi
12       sävellys
13      sijaitsee
15         painaa
17              は
26      developed
43     Literature
45              か
53              か
66              何
112          used
123        Hangul
125           eat
Name: last_token, dtype: object


## 1.2 Binary classification

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(123)

In [16]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 34.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 


In [17]:
!pip install fugashi[unidic-lite]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 583 kB 29.5 MB/s 
     |████████████████████████████████| 47.4 MB 2.2 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=ccea2eacd3e9fcd82918ddd13ca57f151b7bc088dfa00831f7b8144a5d64654c
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [18]:
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [19]:
label_to_ix = {'Non-answerable':0, 'Answerable':1}

In [20]:
df['answerable'] = df.annotations.apply(lambda x: 0 if x['answer_start']==-1 else 1)
df_val['answerable'] = df_val.annotations.apply(lambda x: 0 if x['answer_start']==-1 else 1)

df['answerable_t'] = df.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')
df_val['answerable_t'] = df_val.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

### 1.2.a Design classifier

In [21]:
tokenizer_fi = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
#tokenizer_ja = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese") # Issue
tokenizer_en = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [22]:
class BoWClassifier(nn.Module):

    def __init__(self, num_labels, vocab_size, num_hidden = 2):
        # Calls the init function of nn.Module. 
        super(BoWClassifier, self).__init__()

        # Define the parameters that you need.
        self.linear = nn.Linear(vocab_size, num_hidden)
        # non-linearity (here it is also a layer!)
        self.nonlinear = nn.ReLU()
        # final affine transformation
        self.final = nn.Linear(num_hidden, num_labels)

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return self.final(self.nonlinear(self.linear(bow_vec)))

In [23]:
# Functions to create BoW vectors
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

Test model is running ...

In [24]:
# Function to map each word in the vocab to an unique integer
# Indexing the Bag of words vector
word_to_ix = {}
for sent in df['question_text_t'] + df['document_plaintext_t']:# + df_val['question_text_t'] + df_val['document_plaintext_t']:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [26]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# The first output below is A, the second is b.
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-7.6024e-04,  6.1756e-05, -9.2594e-04,  ...,  1.6241e-03,
         -1.5288e-03,  7.7469e-04],
        [ 1.0381e-03, -1.5680e-03, -1.6714e-03,  ...,  1.2232e-03,
         -1.7472e-03, -3.9226e-04]], requires_grad=True)
Parameter containing:
tensor([ 0.0016, -0.0011], requires_grad=True)
Parameter containing:
tensor([[-0.1705,  0.3914],
        [-0.6458, -0.5898]], requires_grad=True)
Parameter containing:
tensor([-0.0808, -0.0233], requires_grad=True)


In [27]:
test_data = df.loc[df.language == 'finnish',['question_text_t','answerable']].head(10).values.tolist()

In [28]:
with torch.no_grad():
    for text, label in test_data:
        bow_vec = make_bow_vector(text, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# Print the matrix column corresponding to "Charles"
print(next(model.parameters())[:, word_to_ix["Charles"]])

tensor([[-0.0809, -0.0238]])
tensor([[-0.0819, -0.0273]])
tensor([[-0.0802, -0.0253]])
tensor([[-0.0808, -0.0233]])
tensor([[-0.0811, -0.0244]])
tensor([[-0.0808, -0.0233]])
tensor([[-0.0808, -0.0233]])
tensor([[-0.0806, -0.0260]])
tensor([[-0.0811, -0.0244]])
tensor([[-0.0817, -0.0295]])
tensor([ 6.1756e-05, -1.5680e-03], grad_fn=<SelectBackward0>)


### 1.2.b Train classifier

In [29]:
# Model for each language
word_to_ix_fi = tokenizer_fi.vocab
model_fi = BoWClassifier(NUM_LABELS, len(word_to_ix_fi))

word_to_ix_en = tokenizer_en.vocab
model_en = BoWClassifier(NUM_LABELS, len(word_to_ix_en))

#word_to_ix_ja = tokenizer_ja.vocab
#model_ja = BoWClassifier(NUM_LABELS, len(word_to_ix_ja))

In [30]:
df_lang = df.loc[df.language == 'finnish',['question_text_t','answerable_t']].values.tolist()

In [31]:
from torch.optim import Adam
from tqdm import tqdm

In [32]:
epochs = 5
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [33]:
for epoch in range(epochs):
    for instance, label in tqdm(df_lang):
        # Step 1. Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Tensor as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

100%|██████████| 13701/13701 [00:24<00:00, 551.13it/s]


Words not in voc: 

*   special token (oov)
*   BPE better



In [34]:
def train_loop(df, model, loss_fn, optimizer, word_to_ix, label_to_ix):
  for X, y in tqdm(df):
    # Step 1: Clear gradients before instance
    model.zero_grad()

    # Step 2: Make BOW vector and wrap in a Tensor as an integer
    bow_vec = make_bow_vector(X, word_to_ix)
    target = make_target(y, label_to_ix)

    # Step 3: Run forward pass
    pred = model(bow_vec)

    # Step 4: Compute loss, gradients, and update the parameters
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

In [35]:
def test_loop(df, model, loss_fn, word_to_ix, label_to_ix):
  size = len(df)
  #num_batches = len(???)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in tqdm(df):
      bow_vec = make_bow_vector(X, word_to_ix)
      target = make_target(y, label_to_ix)

      pred = model(bow_vec)
      test_loss += loss_fn(pred, target).item()
      correct += (pred.argmax(1) == target).type(torch.float).sum().item()
  
  #test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

In [36]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [37]:
for t in range(epochs):
  print(f"Epoch {t+1}\n--------------")
  train_loop(df_lang, model, loss_function, optimizer, word_to_ix, label_to_ix)
  #test_loop(df_lang, model, loss_function, word_to_ix, label_to_ix)
print('Done!')

Epoch 1
--------------


100%|██████████| 13701/13701 [00:24<00:00, 548.61it/s]


Epoch 2
--------------


100%|██████████| 13701/13701 [00:24<00:00, 553.99it/s]


Epoch 3
--------------


...

Epoch 4
--------------


100%|██████████| 13701/13701 [00:24<00:00, 557.82it/s]


Epoch 5
--------------


100%|██████████| 13701/13701 [00:24<00:00, 559.82it/s]

Done!


Model is not improving...

### 1.2.c Evaluate classifier

In [38]:
test_loop(df_lang, model, loss_function, word_to_ix, label_to_ix)

100%|██████████| 13701/13701 [00:06<00:00, 2090.20it/s]

Test Error: 
 Accuracy: 50.0%


In [39]:
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[0.5805, 0.2568]])
tensor([[0.5797, 0.2566]])
tensor([[0.5797, 0.2566]])
tensor([[0.5806, 0.2570]])
tensor([[0.5795, 0.2574]])
tensor([[0.5782, 0.2562]])
tensor([[0.5798, 0.2579]])
tensor([[0.5798, 0.2566]])
tensor([[0.5807, 0.2568]])
tensor([[0.5807, 0.2568]])


Calculate accuracy

# Assignment 2 
Representation Learning

In [40]:
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords

In [41]:
def get_bpemb_features(dataset, bpemb):
  # With bpemb we can tokenize and embed an entire document using .embed(x)
  X = [bpemb.embed(x).mean(0) for x in tqdm(dataset[:,0])]
  y = list(dataset[:,1])
  return X,y

In [42]:
def get_glove_features(dataset, glove, nlp):
  X = []
  for x in tqdm(dataset[:,0]):
    # For glove embeddings, we first tokenize the sentence using spacy, lower-case and remove stopwords, and get the available word vectors
    vecs = np.vstack([glove[t.text.lower()] if t.text.lower() in glove else np.zeros(100) for t in nlp(x) if t.text.lower() not in stopwords])
    X.append(vecs.mean(0))
  y = list(dataset[:,1])
  return np.stack(X),y